In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install accelerate peft bitsandbytes pip install git+https://github.com/huggingface/transformers trl py7zr auto-gptq optimum
# accelerate for gpu computation
# peft for use lora
# # bitsandbytes for quantization computing
# trl for use sfttrainer
# optimum for fine tuning gptq model


  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-f58b0e5n
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-f58b0e5n
  Resolved https://github.com/huggingface/transformers to commit bc72b4e2cdcbc80d5f56731f35dbc9c18b4c8de6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 100.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.3/403.3

In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments
from trl import SFTTrainer
import os

In [ ]:
data = load_dataset("samsum", split="train")
data

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [ ]:
data_df = data.to_pandas()
data_df

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."
...,...,...,...
14727,13863028,Romeo: You are on my ‘People you may know’ lis...,Romeo is trying to get Greta to add him to her...
14728,13828570,Theresa: <file_photo>\r\nTheresa: <file_photo>...,Theresa is at work. She gets free food and fre...
14729,13819050,John: Every day some bad news. Japan will hunt...,Japan is going to hunt whales again. Island an...
14730,13828395,Jennifer: Dear Celia! How are you doing?\r\nJe...,Celia couldn't make it to the afternoon with t...


In [ ]:
data_df["text"] = data_df[["dialogue", "summary"]].apply(lambda x: "#Human: Summarize this following dialogue: " + x["dialogue"] + "\n#Assistant: " +x["summary"], axis=1)
print(data_df.iloc[0])

id                                                   13818513
dialogue    Amanda: I baked  cookies. Do you want some?\r\...
summary     Amanda baked cookies and will bring Jerry some...
text        #Human: Summarize this following dialogue: Ama...
Name: 0, dtype: object


In [ ]:
data = Dataset.from_pandas(data_df)
data

Dataset({
    features: ['id', 'dialogue', 'summary', 'text'],
    num_rows: 14732
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ")
tokenizer.pad_token = tokenizer.eos_token # the eos_token represents the end of a sequence, and the pad_token is used for padding sequences to a consistent length during tokenization.
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True, tokenizer=tokenizer) #  The extra llama quantizer is a technique used in quantization. By setting it to True, the extra llama quantizer is disabled.
model = AutoModelForCausalLM.from_pretrained(
                          "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",
                          quantization_config=quantization_config_loading,
                          device_map="auto"
                    )

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

id                                                   13818513
dialogue    Amanda: I baked  cookies. Do you want some?\r\...
summary     Amanda baked cookies and will bring Jerry some...
text        ###Human: Summarize this following dialogue: A...
Name: 0, dtype: object


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


In [ ]:
model.config.use_cache=False # setting it for training
model.config.pretraining_tp=1 # replicate the pre-training performance
model.gradient_checkpointing_enable() # Gradient checkpointing is a technique used during training to reduce memory consumption, It can be particularly helpful when dealing with large models that don't fit into GPU memory.
model = prepare_model_for_kbit_training(model) # use for retrive from CUDA out of memory
peft_config = LoraConfig( # holds configuration parameters for a model
    r=16, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM", target_modules=["q_proj", "v_proj"]  # "CAUSAL_LM". This suggests that the model is configured for a Causal Language Modeling task.
) # specific modules ("q_proj" and "v_proj") are the target or focus of the configuration.

In [ ]:
model = get_peft_model(model, peft_config)  # retrieve a model with certain configurations.
training_arguments = TrainingArguments(
  output_dir="/content/drive/MyDrive/Mistral/mistral_model",
  per_device_train_batch_size=8,
  gradient_accumulation_steps=1,
  optim="paged_adamw_32bit", # Choose the optimizer as we used quantizer model
  learning_rate=2e-4,
  lr_scheduler_type="cosine", # cosine function to smoothly decrease and then increase the learning rate
  save_strategy="epoch",
  logging_steps=100,
  num_train_epochs=1,
  max_steps=250,
  fp16=True,
  # push_to_hub=True
)

In [ ]:
trainer = SFTTrainer(
  model=model,
  train_dataset=data,
  peft_config=peft_config,
  dataset_text_field="text",
  args=training_arguments,
  tokenizer=tokenizer,
  packing=False,
  max_seq_length=512
)
trainer.train()

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:284: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
100,1.867700
200,1.772300


TrainOutput(global_step=250, training_loss=1.806415985107422, metrics={'train_runtime': 2689.0365, 'train_samples_per_second': 0.744, 'train_steps_per_second': 0.093, 'total_flos': 669478733217792.0, 'train_loss': 1.806415985107422, 'epoch': 0.14})

# Inference

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Mistral/mistral_model/checkpoint-250")

inputs = tokenizer("""
#Human: Summarize this following dialogue: Vasanth: I'm at the railway station in Chennai Karthik: No problems so far? Vasanth: no, everything's going smoothly Karthik: good. lets meet there soon!
#Assistant: """, return_tensors="pt").to("cuda") # he return_tensors="pt" argument specifies that the tokenizer should return PyTorch tensors.

model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/Mistral/mistral_model/checkpoint-250",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1, # highest probability will be considered
    temperature=0.1, #  controls the randomness of the sampling process
    max_new_tokens=25,
    pad_token_id=tokenizer.eos_token_id
)

In [ ]:
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


#Human: Summarize this following dialogue: Vasanth: I'm at the railway station in Chennai Karthik: No problems so far? Vasanth: no, everything's going smoothly Karthik: good. lets meet there soon!
#Assistant:  Vasanth is at the railway station in Chennai. Everything is going smoothly. He will meet Karthik soon.
